In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
import os
import json
import requests
from pypdf import PdfReader
from IPython.display import display, Markdown
import gradio as gr

In [ ]:
load_dotenv(override = True)
openai = OpenAI()

In [ ]:
pushover_user_key = os.getenv("PUSHOVER_USER_KEY")
pushover_app_token = os.getenv("PUSHOVER_APP_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

if not pushover_user_key or not pushover_app_token:
    raise ValueError("PUSHOVER_USER_KEY and PUSHOVER_APP_TOKEN must be set in environment variables.")

In [ ]:
def push(message):
    print("Sending PushOver Notification...")
    payload = {
        "token": pushover_app_token,
        "user": pushover_user_key,
        "message": message
    }
    requests.post(pushover_url, data=payload)

In [ ]:
push("PushOver Notification Test from LearningLLMCode")

In [ ]:
def record_user_details(email, name="", notes=""):
    push(f"New User Details Recorded:\nEmail: {email}\nName: {name}\nNotes: {notes}")
    return {"recorded": "ok"}

In [ ]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address.",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The user's email address."
            },
            "name": {
                "type": "string",
                "description": "The user's name (optional)."
            },
            "notes": {
                "type": "string",
                "description": "Any additional notes about the user (optional)."
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }

In [ ]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [ ]:
tools

In [ ]:

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        # THE BIG IF STATEMENT!!!

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)

        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [ ]:
globals()["record_unknown_question"]("this is a really hard question")

In [ ]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [ ]:
reader = PdfReader("knowledgebase/Resume-Swapnil Puri_Full_SS.pdf")
candidate_resume = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        candidate_resumeresume += text

with open("knowledgebase/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Swapnil Puri"

In [ ]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and candidate resume which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## Candidate Resume:\n{candidate_resume}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:

        # This is the call to the LLM - see that we pass in the tools json

        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)

        finish_reason = response.choices[0].finish_reason
        
        # If the LLM wants to call a tool, we do that!
         
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat, type="messages").launch()